In [1]:
# !pip install langchain transformers qdrant-client accelerate torch bitsandbytes

In [2]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama2")


In [3]:
llm.invoke("Tell me a joke")

'Why was the math book sad? Because it had too many problems! 😄'

In [4]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
 chunk_size=1000,
 chunk_overlap=20,
 length_function=len,
 is_separator_regex=False,
)
loader = DirectoryLoader('Hindi-Aesthetics-Corpus/Corpus', loader_cls=TextLoader)
docs = loader.load_and_split(text_splitter=text_splitter)

In [9]:
import pandas as pd
data = []
for doc in docs:
 row_data = {
 "page_content": doc.page_content,
 "metadata": doc.metadata
 }
 data.append(row_data)
df = pd.DataFrame(data)
df['page_content'] = df['page_content'].replace('\\n', ' ', regex=True)

In [ ]:
import fasttext as ft
# Loding model for Hindi.
embed_model = ft.load_model('wiki.hi.bin')

In [11]:
df['embeddings'] = df['page_content'].apply(lambda x: (embed_model.get_sentence_vector(x)).tolist())

In [18]:
df['id'] = range(1, len(df) + 1)

In [19]:
payload = df[['page_content', 'metadata']].to_dict(orient='records')

In [ ]:
from qdrant_client import QdrantClient
client = QdrantClient(location=':memory:')

In [20]:
from langchain.vectorstores import Qdrant

In [22]:
Qdrant.from_documents(docs, index_name="hindi_aesthetics_corpus", embedding=df['embeddings'])

AttributeError: 'Series' object has no attribute 'embed_documents'

In [ ]:

from langchain.embeddings import HuggingFaceEmbeddings

import fasttext as ft
# Loding model for Hindi.
embed_model = ft.load_model('wiki.hi.bin')

doc_store = Qdrant.from_texts(
    texts, embeddings, url="<qdrant-url>", api_key="<qdrant-api-key>", collection_name="texts"
)

In [ ]:
# !pip install fasttext
# wget https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.hi.zip
# !unzip wiki.hi.zip

In [6]:
# import fasttext.util
# fasttext.util.download_model('hi', if_exists='ignore')  # English
# ft = fasttext.load_model('cc.hi.300.bin')